In [23]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn import metrics
import statsmodels.formula.api as sm
%matplotlib inline

In [24]:
train = pd.read_csv('../../../Data/train.csv')
test = pd.read_csv('../../../Data/test.csv')

In [25]:
train = train.drop(columns = ['Id'], axis = 1)
test = test.drop(columns = ['Id'], axis = 1)

In [26]:
train = train[train['GrLivArea']<4500]
train.reset_index(drop = True, inplace = True)
train['SalePrice'] = np.log1p(train['SalePrice'])
y = train['SalePrice'].reset_index(drop = True)

In [27]:
train_features = train.drop(columns = 'SalePrice', axis = 1)
test_features = test
features = pd.concat([train_features, test_features]).reset_index(drop = True)

In [28]:
def missing_values(data):
    total = data.isnull().sum().sort_values(ascending = False)
    percent = (data.isnull().sum()/data.isnull().count()*100).sort_values(ascending = False)
    ms = pd.concat([total, percent], axis = 1, keys = ['Total', 'Percent'])
    ms = ms[ms['Percent']>0]
    return ms

In [84]:
missing_values(features)

,Total,Percent


In [38]:
features = features.dropna(thresh = len(features)*0.70,axis =1)

In [83]:
for i in features.select_dtypes(include = np.number):
    features[i].fillna(features[i].mean(), inplace = True)

In [79]:
for i in features.select_dtypes(include = 'object'):
    features[i].fillna(features[i].mode()[0], inplace = True)